In [23]:
function newtmin_BFGS(p, maxIts, optTol)
    # obj is a function that evaluates the objective value, 
    #    gradient and hessian a x.
    # x0 is the starting point.
    # maxIts = maximum interations
    # optTol is the optimality tolerance on the gradient
    
    its = 1;
    Tol = 0;
    err = 0;
    fault_count = 0;
    
    # calculate the gradient from the objective function.  In our case, 
    # these values will be calls to the functions in Toms566
    x_k0 = p.x0;
    n = p.n;
    h_k0 = p.hes(x_k0);
    delta = 1;
    tolFactor = optTol*norm(p.grd(x_k0));
    
    #initial guess of hessian.  Make sure that the matrix is positive definite
    if((det(h_k0)<0) || !(rank(h_k0)==3) )  
        H_diag,H_eigMat = eig(h_k0);
            for j=1:length(H_diag)
                if H_diag[j]<0
                    # Replace all negative eigenvalues with their absolute value
                    H_diag[j]=(abs(H_diag[j]));
                elseif H_diag[j]==0
                    # Replace all zero eigenvalues with delta
                    H_diag[j]=delta;
                end
            end
            B_k0 = H_eigMat*diagm(H_diag)*H_eigMat';
    else  B_k0 = h_k0;     
    end

    # Start the Loop 
    for i=1:maxIts
        # check for positive definiteness and full rank
        g_k0 = p.grd(x_k0);
        
        Tol = norm(g_k0);
        if Tol<tolFactor
            break;
        end
        step_k0 = B_k0\g_k0;
                
        #calculate the step length ----Armijo Backtracking 
        alpha = 1;
        while( p.obj(x_k0-alpha*step_k0) > ( (p.obj(x_k0)) ) )
            alpha=.5*alpha;
            if alpha < ( 1e-4)
                break;
            end
        end
        
        x_k1 = x_k0 - alpha*(step_k0);
        g_k1 = p.grd(x_k1);
        y = g_k1 - g_k0;
        s = x_k1 - x_k0;
        
        #BFGS Update
        dB = (y*y')/dot(s,y) - (B_k0*(s*s')*B_k0)/dot(s,B_k0*s);
        
        B_k0 = B_k0 + dB;
        
        x_k0 = x_k1;
        its+=1;
 
    end
    x = x_k0;
    f = p.obj(x);
    B = B_k0;
 
    return(f,its,Tol,B,x);
end

newtmin_BFGS (generic function with 1 method)

In [2]:
Pkg.test("Toms566")

INFO: Testing Toms566


No.  Name                             n        f(x0)     |∇f(x0)| cond(∇²f(x0))
  

INFO: Toms566 tests passed


1  Hellical valley                  3     2.50e+03     1.88e+03     9.82e+00
  2  Bigg's EXP6                      6     7.79e-01     2.55e+00     1.30e+01
  3  Gaussian                         3     3.89e-06     7.45e-03     5.10e+01
  4  Powell                           2     1.14e+00     2.00e+04     1.37e+08
  5  Box 3-dim                        3     1.03e+03     1.49e+02     1.19e+02
  6  Variably dimensioned            40     9.39e+10     1.01e+11     1.65e+04
  7  Watson                           9     3.00e+01     1.78e+02     1.63e+02
  8  Penalty I                       60     5.45e+09     8.02e+07     4.24e+00
  9  Penalty II                      65     2.87e+05     3.28e+05     9.84e+01
 10  Brown badly scaled               2     1.00e+12     2.00e+06     1.00e+00
 11  Brown and Denis                  4     7.93e+06     2.14e+06     6.93e+02
 12  Gulf research and development    3     1.21e+01     3.97e+01     4.65e+04
 13  Trigonometric                   40     2.01e-03  

In [4]:
using Toms566

In [25]:
p = Problem(2)
f,its,Tol,B,x = newtmin_BFGS(p, 2000, 1e-8)

(5.235590508522748e-18,124,5.282324573060972e-9,
6x6 Array{Float64,2}:
  2.4526    -0.141421  -3.62049    0.129759   0.904946  -0.740501
 -0.141421   0.119284   0.736984  -0.183717  -0.211377   0.435184
 -3.62049    0.736984   8.17384   -0.979152  -2.28805    3.02939 
  0.129759  -0.183717  -0.979152   0.32094    0.253511  -0.661753
  0.904946  -0.211377  -2.28805    0.253511   0.692059  -0.872698
 -0.740501   0.435184   3.02939   -0.661753  -0.872698   1.63114 ,

[1.0000000548300119,9.99999983452826,1.000000085501594,5.000000191065231,4.000000228835369,3.000000108356552])